In [ ]:
#! pip install tensorflow

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [2]:
image_train = Path('C:/Users/vasan/OneDrive/EmissionsDatathon/Dataset/Data/Train')

dir_name = r'C:\Users\vasan\OneDrive\EmissionsDatathon\Zeroing Methane Emissions - Dataset'
filename_suffix = 'csv'

In [3]:
filepaths = pd.Series(list(image_train.glob(r'**/*.png')), name='Filepath').astype(str)

In [4]:
filepaths.head()

0    C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
1    C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
2    C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
3    C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
4    C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
Name: Filepath, dtype: object

In [5]:
df_filepaths = filepaths.to_frame()

In [6]:
df_filepaths

,Filepath
0,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
1,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
2,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
3,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
4,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
...,...
3758,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
3759,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
3760,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...
3761,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...


In [7]:
res = os.listdir(image_train)
df_filepaths['Name'] = pd.DataFrame (res, columns = ['FileName'])

In [8]:
df_filepaths['candidate_id'] = [x.split('_')[-0] for x in df_filepaths['Name']]

In [9]:
df_filepaths.to_excel("output.xlsx")  

In [10]:
skiprows = 0
#Means read in the ',' as thousand seperator. Also drops all columns which are unnamed.
df = pd.read_csv("permian_plume_list_2019 Jeremy Zhao.csv", thousands=',', skiprows = skiprows)
#df = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
df.head()

,Unnamed: 0,source_id,plume_lat,plume_lon,candidate_id,date,time,qplume,sigma_qplume,active_flair_detected,inactive_flair_detected
0,0,P00001,32.212063,-103.697076,ang20190922t192642-2,2019-09-22,19:26:42,2579.528100,902.732655,False,False
1,1,P00002,32.200072,-103.673301,ang20190922t192642-4,2019-09-22,19:26:42,420.725207,147.155782,False,False
2,2,P00003,32.116008,-103.626345,ang20190922t192642-5,2019-09-22,19:26:42,1051.909600,339.267587,False,False
3,3,P00004,32.322594,-103.816072,ang20190922t194340-1,2019-09-22,19:43:40,1736.644250,1035.427670,False,False
4,4,P00005,32.284083,-103.793989,ang20190922t194340-2,2019-09-22,19:43:40,870.605019,435.968147,False,False


In [11]:
print(df.shape)
print(df_filepaths.shape)

(3067, 11)
(3763, 3)


In [12]:
df2 = pd.merge(df_filepaths, df, on='candidate_id')

In [13]:
print(df2.shape)

(3067, 13)


In [14]:
df2.head()

,Filepath,Name,candidate_id,Unnamed: 0,source_id,plume_lat,plume_lon,date,time,qplume,sigma_qplume,active_flair_detected,inactive_flair_detected
0,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...,ang20190922t192642-2_r4578_c217-plume.png,ang20190922t192642-2,0,P00001,32.212063,-103.697076,2019-09-22,19:26:42,2579.528100,902.732655,False,False
1,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...,ang20190922t192642-4_r4928_c373-plume.png,ang20190922t192642-4,1,P00002,32.200072,-103.673301,2019-09-22,19:26:42,420.725207,147.155782,False,False
2,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...,ang20190922t192642-5_r6423_c113-plume.png,ang20190922t192642-5,2,P00003,32.116008,-103.626345,2019-09-22,19:26:42,1051.909600,339.267587,False,False
3,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...,ang20190922t192642-A_r3270_c384-plume.png,ang20190922t192642-A,759,P00156,32.283447,-103.741815,2019-09-22,19:26:42,296.676732,98.755811,False,False
4,C:\Users\vasan\OneDrive\EmissionsDatathon\Data...,ang20190922t192642-B_r4034_c360-plume.png,ang20190922t192642-B,760,P00119,32.244361,-103.711379,2019-09-22,19:26:42,640.968143,190.138173,False,False


In [15]:
df2['candidate_id'].nunique()

3067

In [16]:
def detect_outliers_zscore(data, threshold=3):
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    num_outliers = np.sum(z_scores > threshold)
    return num_outliers

In [17]:
detect_outliers_zscore(df2['qplume'], threshold=3)

49

In [18]:
def drop_outliers_zscore(data, threshold=3):
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    return data[z_scores <= threshold]

In [19]:
df2.to_excel("output2.xlsx")  

In [20]:
drop_outliers_zscore(df2.qplume, threshold=3)

0       2579.528100
1        420.725207
2       1051.909600
3        296.676732
4        640.968143
           ...     
3062     244.219726
3063     185.375804
3064     237.801830
3065     282.037503
3066     174.266195
Name: qplume, Length: 3018, dtype: float64

In [21]:
df2.shape

(3067, 13)

In [22]:
train_df, test_df = train_test_split(df2, train_size=0.7, shuffle=True, random_state=1)

In [23]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
# https://www.kaggle.com/code/gcdatkin/age-prediction-from-images-cnn-regression

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='qplume',
    target_size=(217, 217),
    color_mode='rgba',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

In [ ]:
train_images

In [ ]:
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='qplume',
    target_size=(217, 217),
    color_mode='rgba',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

In [ ]:
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='qplume',
    target_size=(217, 217),
    color_mode='rgba',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

In [ ]:
inputs = tf.keras.Input(shape=(217, 217, 4))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
predicted_ages = np.squeeze(model.predict(test_images))
true_ages = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_ages, predicted_ages)
print("Test R^2 Score: {:.5f}".format(r2))

In [ ]:
null_rmse = np.sqrt(np.sum((true_ages - np.mean(true_ages))**2) / len(true_ages))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

# 2nd Try

In [ ]:
def image_processing(fil):
#Open image and import it as a numpy array
    with open(fil, 'rb') as f:
        im1 = f.read()
    im1_data = np.frombuffer(im1, dtype='uint8')
#Read the numpy arrays as color images in OpenCV
    image_bgr2 = cv2.imdecode(im1_data, cv2.IMREAD_COLOR)
#Convert to HSV for creating a mask
    image_hsv = cv2.cvtColor(image_bgr2, cv2.COLOR_BGR2HSV)
#Convert to grayscale that will actually be used for training, instead of color image 
    image_gray = cv2.cvtColor(image_bgr2, cv2.COLOR_BGR2GRAY)
#Create a mask that detects the red rectangular tags present in each image
    mask = cv2.inRange(image_hsv, (0,255,255), (0,255,255))
#Get the coordinates of the red rectangle in the image, 
    #But take entire image if mask fails to detect the red rectangle
    if len(np.where(mask != 0)[0]) != 0:
        y1 = min(np.where(mask != 0)[0])
        y2 = max(np.where(mask != 0)[0])
    else:
        y1 = 0                                     
        y2 = len(mask)
    if len(np.where(mask != 0)[1]) != 0:
        x1 = min(np.where(mask != 0)[1])
        x2 = max(np.where(mask != 0)[1])
    else:
        x1 = 0
        x2 = len(mask[0])
#Crop the grayscle image along those coordinates
    image_cropped = image_gray[y1:y2, x1:x2]
    if image_cropped.size ==0:
        print(fil)
        return image_cropped
    else:
    #Resize the image to 100x100 pixels size
        image_128x128 = cv2.resize(image_cropped, (128, 128))
#Save image as in form of array of 10000x1
        image_arr = image_128x128.flatten()
    return image_arr

In [ ]:
from tqdm import tqdm

In [ ]:
image_list = []

In [ ]:
for file in tqdm(df2['Filepath'].tolist()[:]):
    image_list.append(image_processing(file))

In [ ]:
X = np.array(image_list)

In [ ]:
np.save('processed_128x128_image.npy',X/255,allow_pickle=True)

In [ ]:
pic_array = np.load('processed_128x128_image.npy',allow_pickle=True)

In [ ]:
#remove non 16384 dimension out of the numpy array
X =[]
exclude =[]
for i in range(len(pic_array)):
    if pic_array[i].shape == (16384,):
        X.append(pic_array[i])
    else:
        exclude.append(i)
X =np.array(X)
#also remove from the dataframe
df2.drop(df2.index[exclude],inplace=True)

In [ ]:
np.random.seed(17)
for i in np.random.randint(0, len(pic_array), 2):
    plt.figure()
    plt.imshow(pic_array[i].reshape(128, 128), cmap='gray'), plt.axis('off')

In [ ]:
import numpy as np #for numerical computations
import pandas as pd #for dataframe operations

from matplotlib import pyplot as plt #for viewing images and plots
%matplotlib inline 
#So that Matplotlib plots don't open in separate windows outside the notebook

import urllib #For fetching data from Web URLs

import cv2   #For image processing

from sklearn.preprocessing import LabelEncoder    #For encoding categorical variables
from sklearn.model_selection import train_test_split #For splitting of data
#All tensorflow utilities for creating, training and working with a CNN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
Y = np.array(df2.qplume.tolist())

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
#Segregation of a test set for testing on the trained model

#X_test = X[2095:,]
#Y_test2 = Y[2095:,]

#Seperation of a validation set from the remaing training set (required for validation while training)

#X_train, X_val, Y_train, Y_val = train_test_split(X[:2094,], Y[:2094,], test_size=0.15, random_state=13)

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Split the train set into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=42)

In [ ]:
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 1)

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

X_val.shape

In [ ]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
#Define a Convolutional Neural Network Model

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = input_shape))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
#model.add(Dense(n_classes, activation='softmax'))
model.add(Dense(1, activation='relu'))
learning_rate = 0.001

model.compile(loss = 'mse',
              optimizer = Adam(learning_rate))

model.summary()

In [ ]:
save_at = "model_regression.hdf5"
save_best2 = ModelCheckpoint (save_at, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='max')

In [ ]:
#Train the CNN

history = model.fit( X_train, Y_train, 
                    epochs = 15, batch_size = 100, 
                    callbacks=[save_best2], verbose=1, 
                   validation_data = (X_val, Y_val))

In [ ]:
plt.figure(figsize=(6, 5))
plt.plot(history.history['loss'], color='r')
plt.plot(history.history['val_loss'], color='g')
plt.show()

In [ ]:
Y_pred = np.round(model.predict(X_test))

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# Calculate the mean squared error
mse = mean_squared_error(Y_test, Y_pred)

# Calculate the root mean squared error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

r2 = r2_score(Y_test, Y_pred)
print("Test R^2 Score: {:.5f}".format(r2))

# Try 3

In [ ]:
def unet(pretrained_weights = None, input_size = (128,128,1)):

    inputs = Input(input_size)
    
    # 128
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # 128 -> 64
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # 64 -> 32
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # 32 -> 16
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Center: 16 -> 8
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)
    drop5 = Dropout(0.5)(conv5)

    # 8 -> 16
    up6 = Conv2DTranspose(256, (2,2),strides=(2,2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop5)
    up6 = BatchNormalization()(up6)
    merge6 = concatenate([drop4,up6],axis=3)
    conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    conv6 = BatchNormalization()(conv6)
    
    # 16 -> 32
    up7 = Conv2DTranspose(128,(2,2),strides=(2,2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    up7 = BatchNormalization()(up7)
    merge7 = concatenate([conv3,up7],axis=3)
    conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    conv7 = BatchNormalization()(conv7)
    
    # 32 -> 64
    up8 = Conv2DTranspose(64,(2,2),strides=(2,2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    up8 = BatchNormalization()(up8)
    merge8 = concatenate([conv2,up8],axis=3)
    conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv8 = BatchNormalization()(conv8)
    
    # 64 -> 128
    up9 = Conv2DTranspose(32,(2,2),strides=(2,2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    up9 = BatchNormalization()(up9)
    merge9 = concatenate([conv1,up9],axis=3)
    conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [ ]:
save_at2 = "model_regression2.hdf5"
save_best2 = ModelCheckpoint (save_at, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='max')

In [ ]:
early_stopping2 = EarlyStopping(monitor ='val_accuracy', patience = 5, verbose = 1)
model_checkpoint2 = ModelCheckpoint(save_at2, monitor='val_accuracy', save_best_only = True, verbose = 1)

In [ ]:
#Train the CNN

history = model.fit( X_train, Y_train, 
                    epochs = 30, batch_size = 32, 
                    callbacks = [early_stopping2, model_checkpoint2], verbose=1, 
                   validation_data = (X_val, Y_val))